In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
import psycopg
import sqlite3
import plotly.express as px
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sqlalchemy import create_engine    
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
postgrespassword = os.getenv('POSTGRES_PASSWORD')

In [2]:
ct = contrans()

In [3]:
server, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD)

In [4]:
members = ct.get_bioguideIDs()

In [6]:
terms_df, members = ct.terms_df(members)

In [8]:
members = ct.make_cand_table(members)

In [9]:
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,partyletter,DistIDRunFor,lastname,firstname,name2,CID
0,W000829,08,"Wied, Tony",Republican,WI,2024-11-20T19:56:11Z,https://api.congress.gov/v3/member/W000829?for...,Image courtesy of the Member,https://www.congress.gov/img/member/6734b6724c...,R,WI08,Wied,Tony,Wied (R),NaN
1,L000605,18,"Lee Carter, Erica",Democratic,TX,2024-11-20T19:54:42Z,https://api.congress.gov/v3/member/L000605?for...,Image courtesy of the Member,https://www.congress.gov/img/member/6734b97f4c...,D,TX18,Lee Carter,Erica,Lee Carter (D),NaN
2,G000578,01,"Gaetz, Matt",Republican,FL,2024-11-14T18:50:13Z,https://api.congress.gov/v3/member/G000578?for...,Congressional Pictorial Directory,https://www.congress.gov/img/member/116_rp_fl_...,R,FL01,Gaetz,Matt,Gaetz (R),N00039503
3,N000188,01,"Norcross, Donald",Democratic,NJ,2024-11-05T20:52:08Z,https://api.congress.gov/v3/member/N000188?for...,Image courtesy of the Member,https://www.congress.gov/img/member/n000188_20...,D,NJ01,Norcross,Donald,Norcross (D),N00036154
4,Y000064,S,"Young, Todd",Republican,IN,2024-11-04T13:42:35Z,https://api.congress.gov/v3/member/Y000064?for...,Image courtesy of the Member,https://www.congress.gov/img/member/y000064_20...,R,INS,Young,Todd,Young (R),N00030670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,H001038,26,"Higgins, Brian",Democratic,NY,2024-05-07T18:43:22Z,https://api.congress.gov/v3/member/H001038?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/h001038_20...,D,NY26,Higgins,Brian,Higgins (D),N00027060
550,S001222,03,"Santos, George",Republican,NY,2024-03-04T13:52:36Z,https://api.congress.gov/v3/member/S001222?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001222_20...,R,NY03,Santos,George,Santos (R),N00046427
551,S001192,02,"Stewart, Chris",Republican,UT,2024-02-26T19:36:16Z,https://api.congress.gov/v3/member/S001192?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001192_20...,R,UT02,Stewart,Chris,Stewart (R),N00033932
552,F000062,S,"Feinstein, Dianne",Democratic,CA,2023-10-19T16:13:13Z,https://api.congress.gov/v3/member/F000062?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/f000062_20...,D,CAS,Feinstein,Dianne,Feinstein (D),N00007364


In [11]:
ideology = ct.get_ideology()

In [12]:
ct.make_members_df(members, ideology, engine)

In [13]:
myquery = '''
SELECT *
FROM members
'''
pd.read_sql_query(myquery, con=engine)

,bioguideid,district,name,partyname,state,updatedate,url,depiction_attribution,depiction_imageurl,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,W000829,08,"Wied, Tony",Republican,WI,2024-11-20T19:56:11Z,https://api.congress.gov/v3/member/W000829?for...,Image courtesy of the Member,https://www.congress.gov/img/member/6734b6724c...,R,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,L000605,18,"Lee Carter, Erica",Democratic,TX,2024-11-20T19:54:42Z,https://api.congress.gov/v3/member/L000605?for...,Image courtesy of the Member,https://www.congress.gov/img/member/6734b97f4c...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,G000578,01,"Gaetz, Matt",Republican,FL,2024-11-14T18:50:13Z,https://api.congress.gov/v3/member/G000578?for...,Congressional Pictorial Directory,https://www.congress.gov/img/member/116_rp_fl_...,R,...,None,0.593,-0.643,-166.22574,0.84500,987.0,75.0,None,0.544,-0.700
3,N000188,01,"Norcross, Donald",Democratic,NJ,2024-11-05T20:52:08Z,https://api.congress.gov/v3/member/N000188?for...,Image courtesy of the Member,https://www.congress.gov/img/member/n000188_20...,D,...,None,-0.419,0.340,-50.96336,0.95060,1006.0,23.0,None,-0.391,0.370
4,Y000064,S,"Young, Todd",Republican,IN,2024-11-04T13:42:35Z,https://api.congress.gov/v3/member/Y000064?for...,Image courtesy of the Member,https://www.congress.gov/img/member/y000064_20...,R,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,H001038,26,"Higgins, Brian",Democratic,NY,2024-05-07T18:43:22Z,https://api.congress.gov/v3/member/H001038?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/h001038_20...,D,...,None,-0.348,-0.032,-37.78669,0.94548,674.0,17.0,None,-0.333,-0.035
550,S001222,03,"Santos, George",Republican,NY,2024-03-04T13:52:36Z,https://api.congress.gov/v3/member/S001222?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001222_20...,R,...,None,0.473,-0.421,-117.88222,0.81608,580.0,47.0,None,0.473,-0.420
551,S001192,02,"Stewart, Chris",Republican,UT,2024-02-26T19:36:16Z,https://api.congress.gov/v3/member/S001192?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001192_20...,R,...,None,0.499,0.167,-20.09721,0.94194,336.0,9.0,None,0.437,0.141
552,F000062,S,"Feinstein, Dianne",Democratic,CA,2023-10-19T16:13:13Z,https://api.congress.gov/v3/member/F000062?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/f000062_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [6]:
r =ct.summarize_news('N000188', engine)

In [8]:
r.json()

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [31]:
news_query = 'Tim Kaine congress'
r = requests.get('https://newsapi.org/v2/everything',
            params={'q': news_query, 'apiKey': ct.newskey, 'searchIn': 'content'},
            headers={'User-Agent': ct.useragent})
r.json()

{'status': 'ok',
 'totalResults': 32,
 'articles': [{'source': {'id': 'breitbart-news', 'name': 'Breitbart News'},
   'author': 'Bradley Jaye, Bradley Jaye',
   'title': 'Tim Kaine Projected to Win Reelection to Senate',
   'description': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\nThe post Tim Kaine Projected to Win Reelection to Senate appeared first on Breitbart.',
   'url': 'https://www.breitbart.com/politics/2024/11/06/tim-kaine-projected-to-win-reelection-to-senate/',
   'urlToImage': 'https://media.breitbart.com/media/2024/11/tim-paine-640x335.jpg',
   'publishedAt': '2024-11-06T08:06:54Z',
   'content': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\r\nKaines seat was considered relatively safe, although Republicans were… [+2104 chars]'},
  {'source': {'id': 'new-york-magazine', 'name': 'New York Magazine'},
  

In [36]:
content = [x['title']   for x in r.json()['articles']]

In [37]:
input_text = ' '.join(content)

In [38]:
ct.get_summary_text(input_text)

"Tim Kaine Projected to Win Reelection to Senate Trump Can Prosecute Anybody He Wants, Transition Leader Says Why That Kamala Harris Saturday Night Live Appearance Triggered an FCC Response Links 10/23/2024 Columbia-Class and Virginia-Class Submarines: Overbudget and Behind Schedule (Why?)LIVE: Northern Virginia and statewide election results In Virginia, a Vietnamese American Community Divided Falls Church elected leaders wary of Trump return, GOP dominance in Congress Morning Notes for October 25, 2024 Franks: Dems fumbled by passing over Hillary for Harris Trump’s mainstream picks for top foreign policy posts could reassure allies — and worry China An hour-by-hour guide to watching the races on election night Trump's mainstream picks for top foreign policy posts could reassure allies — and worry China Trump's mainstream picks for top foreign policy posts could reassure allies - and worry China Sen. Tim Kaine wins 3rd term in Washington Rep. Gerry Connolly reveals cancer treatment af